In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

## 1. URL von allen Hauptseiten holen

In [2]:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [18]:
df_v

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,Postulat von Luzian Franzini und Esther Haas b...,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Interpellation von Fabio Iten, Laura Dittli un...",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Postulat von Tabea Zimmermann Gibson, Stéphani...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,Interpellation der SVP-Fraktion betreffend mob...,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig
...,...,...,...,...,...,...,...,...,...
1215,1215,1946,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,NaN,Motion,abgeschlossen
1216,1216,1952,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,NaN,Motion,abgeschlossen
1217,1217,1968,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,NaN,Motion,abgeschlossen
1218,1218,1969,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


In [4]:
df_g=pd.DataFrame(eingang_list)

NameError: name 'eingang_list' is not defined

In [ ]:
df_g.to_csv("KRFrames/df_g_LinkszuPDF")

In [5]:
df_g=pd.read_csv("KRFrames/df_g_LinkszuPDF")

In [6]:
df_g

,Unnamed: 0,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,0,3020,2019-10-11,/dokumente/8316/3020-1-16168_Racial-Profiling.pdf
1,1,3019,2019-10-11,/dokumente/8317/3019-1-16167_Chancengleichheit...
2,2,3018,2019-10-07,/dokumente/8310/3018-1-16166_Praktikum.pdf
3,3,3017,2019-10-06,/dokumente/8311/3017-1-16165_Frauenwahl-stimmr...
4,4,3014,2019-09-26,/dokumente/8312/3014-1-16159_Geschwindigkeitsk...
...,...,...,...,...
1215,1215,762,NaN,/dokumente/4232/pdoc_1047_1.pdf
1216,1216,666,NaN,/dokumente/4215/pdoc_55_1.pdf
1217,1217,304,NaN,/dokumente/4179/pdoc_50_1.pdf
1218,1218,2635,1995-08-07,/gast/geschaefte/1620


In [7]:
#Ich suche nun alle raus, die einen Referenz haben, vielleicht geben die mehr aufschluss auf 
# Wahlvorschläge 
df_ref=df_g[df_g["Link Vorstoss-PDF"].str.contains("gast")]

In [8]:
df_rf=df_ref.rename(columns={"Link Vorstoss-PDF":"url mir Referenz"})

In [10]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_rf["url mir Referenz"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

In [11]:
len(url_liste_ganz)

127

In [12]:
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("KRRef/Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1

KeyboardInterrupt: 

In [13]:
geschaefts_liste=[]#####ACHTUNG ich mache den RANGE kleiner, um weniger GEschäfte zu erhalten.

for seite_g in range(0,126): #Range für alle Geschäfte wäre 1219!
    gesch= "KRRef/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [14]:
# Die fehlerhaften Geschäfte weil sie REfernzen haben abrufen und den PDF-LInk holen
# Da ich irgendwie nich alles in einen Code reinbringe, mach ich mal listen
eingang_list=[]
minuszwei=[]
minusdrei=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    
    
    td_g_list=tr_g_list[-2].find_all("td")
    if td_g_list[1].find_all("a")[0].text== "1":
        minuszwei.append(geschaeft)
    else:
        minusdrei.append(geschaeft)
          

In [ ]:
minuszwei

In [ ]:
len(minusdrei)

In [ ]:
link_minuszwei=[]


file = open('KRRef/Geschaeft6.html', 'r')
text = file.read()
soup_g = BeautifulSoup(text, 'html.parser')
tr_g_list1=soup_g.find_all("tr")
gesch_nr = tr_g_list1[-2].find_all("td")[0].text
td_g_list1=tr_g_list1[-2].find_all("td")
url_v_pdf1=td_g_list1[1].find("a")["href"]
    
minidict21={"Url PDF":url_v_pdf1, "VorlageNR":gesch_nr}
        
link_minuszwei.append(minidict21)

In [ ]:
link_minuszwei

In [ ]:
td_g_list[1].find("a")["href"]

In [15]:
links_minuszwei=[]

for geschaeft in minuszwei:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    gesch_nr = tr_g_list[-2].find_all("td")[0].text
    td_g_list=tr_g_list[-2].find_all("td")
    url_v_pdf=td_g_list[1].find("a")["href"]
    
    minidict2={"Url PDF":url_v_pdf, "VorlageNR":gesch_nr}
        
    links_minuszwei.append(minidict2)

In [19]:
links_minuszwei
#Irgendetwas ist komisch. zum einen die doppelte LInks, zum Andern sind vieles definitiv nicht
#die Vorlage,die ich brauche. Könnten auch TEil der Referenz sein. 

[{'Url PDF': '/dokumente/7354/2881-1-15800_Mutterschaft.pdf',
  'VorlageNR': '2881'},
 {'Url PDF': '/dokumente/8174/2993-1-16111_Mutterschaft.pdf',
  'VorlageNR': '2993'},
 {'Url PDF': '/dokumente/7303/2876-1-15783-Paradiese-Papers.pdf',
  'VorlageNR': '2876'},
 {'Url PDF': '/dokumente/7310/2877-1-15784_Paradiese-Papers.pdf',
  'VorlageNR': '2877'},
 {'Url PDF': '/dokumente/7064/2831-1-15684_Busverbindung.pdf',
  'VorlageNR': '2831'},
 {'Url PDF': '/dokumente/7086/2834-1-15686_OeffentlicherVerkehr.pdf',
  'VorlageNR': '2834'},
 {'Url PDF': '/dokumente/6839/2792-1-15602_KA-Bostadel.pdf',
  'VorlageNR': '2792'},
 {'Url PDF': '/dokumente/6925/2808-1-15622_Konzerte.pdf', 'VorlageNR': '2808'},
 {'Url PDF': '/dokumente/6735/2781-1-15562_2782-1-15563_Bushaltestellen.pdf',
  'VorlageNR': '2781'},
 {'Url PDF': '/dokumente/6736/2781-1-15562_2782-1-15563_Bushaltestellen.pdf',
  'VorlageNR': '2782'},
 {'Url PDF': '/dokumente/6744/2773-1-15558_KA-Geb%C3%A4udeversicherung.pdf',
  'VorlageNR': '2773'

In [ ]:
   try: 
        if tr_g_list[-2].find_all("td")[1].text == "1"== True:
            td_list= tr_g_list[-2].find_all("td")[1]
            url_v_pdf=td_list.find("a")["href"]
    except IndexError:
        continue
        
        
          minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    
            
    eingang_list.append(minidict_g)

In [ ]:
tr_g_list[-3].find_all("td")[1]

In [ ]:
    file = open("KRRef/Geschaeft"+str(3)+".html", 'r')
    text = file.read()
    soup_gneu = BeautifulSoup(text, 'html.parser')
    tr_g_listneu=soup_gneu.find_all("tr")

In [ ]:
tr_g_listneu[-2]

In [ ]:
td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
   
   
     #Da es zum Teil Referenzen hat und dadurch ein anderer Ort abgegriffen werden muss, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    
    elif td_g_list[-1].find_all("a")[0].text== "2": #bei älteren GEschäften sind es Bericht&Anträge
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else: 
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
   
            
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [ ]:
df_rf

In [ ]:
# Nun zusammenbringen mit dem URL
df_referenzen=pd.merge(df_v,df_ref, on="VorlageNR", how="inner")

In [ ]:
df_referenzen

# 3. Die PDFs verlorenen Vorstösse runterladen

In [ ]:
###FALSCH ich muss ja die Links erst noch finden!!!

In [ ]:
for link in tqdm(df_referenzen["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+"REF"+name, "wb") as f:
        f.write(r.content)

# 4. PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [ ]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [ ]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [ ]:
len(pdf_files_list) #Ich habe ja die PDFs ignoriert, zu welchen ich auf Grund der Referenz keine PDF-Links hatte.

In [ ]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr, "Filename":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [ ]:
df_p=pd.DataFrame(dat_pdf_list)

In [ ]:
df_p.sort_values("Einreichedatum") #Stichproben haben gezeigt, dass die fehlerhaften Datumsangaben in GEschäftern sind,
# Welche selber meist schon ein Datum haben. 

In [ ]:
df_p.to_csv("KRFrames/INFOSausPDF")

In [ ]:
df_p["Filename"]

In [ ]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")
df_p

## 4.1 Zeit in Datetime umwandeln
schwierigkeit: wie bringe ich das deutsche Zeitformat in englisches. Mache ich es falsch oder gibts wieder probleme mit den Grossbuchstaben der Zeitformate? Hatten wir windowsuser auch schon.

In [ ]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE') # hier die Sprachwahl
df_p["Einreichedatum"] = pd.to_datetime(df_p["Einreichedatum"], format= "%d. %B %Y", errors='coerce') #Angabe des Formats

In [ ]:
 df_p #Halleluia:-) habs doch noch geschafft. Und die wenigen, die falsch formiert sind, werden nicht auffallen.

In [ ]:
df_p.to_csv("KRFrames/KonvertierteDatenformate")

In [ ]:
df_p=pd.read_csv("KRFrames/KonvertierteDatenformate")

In [ ]:
#Nun noch ohne.0 hinten bei der VorlageNR. Ah es ist ein Float. Kann es also zu int oder str machen
df_p["VorlageNR"]=df_p["VorlageNR"].astype(str)

In [ ]:
def splitting(elem):
    elem = elem.split('.')[0].strip()
    return elem


In [ ]:
df_p["VorlageNR"]=df_p["VorlageNR"].apply(splitting) # So, nun sind die .0 weg. Mal schauen. Das String-Format reicht wohl
del df_p["Unnamed: 0"] # die beiden Spalten entfernen
del df_p["Unnamed: 0.1"]
df_ps=df_p.rename(columns={"Filename": "Link Vorstoss-PDF"}) #Spalte gleich benennen wie im anderen Frame

In [ ]:
df_ps.to_csv("KRFrames/df_p_Datum") # und speichern

## 4.2 Ein Dataframe machen, mit den PDF-Dokumentennamen
- splitting funktion schreiben
- auf Spalte anwenden
- neu abspeichern 

In [ ]:
def splittpdf(elem):
    elem = elem.split('/')[-1].strip()
    return elem

In [ ]:
df_g["Link Vorstoss-PDF"]=df_g["Link Vorstoss-PDF"].apply(splittpdf)

In [ ]:
df_g.to_csv("KRFrames/df_g_Datum_mit_PDF")

## 4.3 Nun zusammenfügen der beiden Daten-Dataframes anhand der PDF-Namen

In [ ]:
df_g1=pd.read_csv("KRFrames/df_g_Datum_mit_PDF")
df_p1=pd.read_csv("KRFrames/df_p_Datum")

In [ ]:
df_dat_max=pd.merge(df_g1, df_p1, how="left",on= "Link Vorstoss-PDF")

In [ ]:
df_dat_max.shape

In [ ]:
df_dat_max.sort_values("Einreichedatum_x").tail(100)  #So, jetzt sind zwar beide zusammen, aber noch nicht vereint
#Jetzt noch irgendwie die beiden Einreichedaten miteinander ableichen/ergängen.

### 4.3.1 die Datenspalten zusammenbekommen.

In [ ]:
#hier füge ich sie zusammen
df_dat_zus=df_dat_max.Einreichedatum_x.combine_first(df_dat_max.Einreichedatum_y)

In [ ]:
df_dat_max['Einreichedatum'] = df_dat_zus # hier füge ich die neu enstandene "Series" dem Dataframe an

In [ ]:
df_datum=df_dat_max[["Unnamed: 0_x","VorlageNR_x", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_datum.shape

In [ ]:
df_datum["Einreichedatum"].sort_values().tail(30)
                                             

In [ ]:
df_datum.to_csv("KRFrames/df_datum_ALLE")

# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [ ]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_mit_Namen_und_Parteien")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



In [ ]:
#  zuerst vorputzen
df_v1 = df_v[["VorlageNR", "Geschäft"]].copy()
df_v1

In [ ]:
df_dat_all=df_dat_all.rename(columns={"VorlageNR_x": "VorlageNR"})

In [ ]:
df_dat_all

In [ ]:
# Nun zusammenfügen
df_alles=pd.merge(df_v1, df_dat_all, how="left",on= "VorlageNR")

In [ ]:
df_alles.to_csv("KRFrames/df_alles_ohne_Namen")

In [ ]:
df_np

In [ ]:
df_alles_ganz=pd.merge(df_alles, df_np, how="left", on="VorlageNR")
df_alles_ganz

In [ ]:
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_tuti #Wieso wird es nicht ergänzt. Der Fehler ist im File 02 Namen passiert. dort nachschauen.

In [ ]:
df_tuti["Einreichedatum"].sort_values().tail(80)
# Obwohl mir nur von 30 Geschäften das Datum fehlt, explodiert dies, wenn es auf die Namen-Tabelle kommt.

In [ ]:
df_tuti.to_csv("KRFrames/df_tuti_Ganzes_Frame")

# Daten aus B&A mit falschen Zahlen
Eine Lösung wäre, nochmals die Links zu den PDF auslesen und diese in zwei verschiedene Listen verpacken. 
Da die Datenmigration im Jahr 2000 war, schaue ich einfach mal die Zahlen ab 2005 an. Vo dort an, werden die meisten Geschäfte tatsächlich neu sein. 
## #Andere Variante: ich lasse die B&A einfach weg. Da gibt es eh keine verlässliche Daten. Mengenmässig.
# Nun plotten

In [ ]:
df_tuti = df_tuti.set_index('Einreichedatum')

In [ ]:
df_tuti

In [ ]:
df_tuti.dtypes